In [ ]:
"""
import packages
"""
import os, glob
import json
import pandas as pd
import numpy as np
import math, random
from datetime import datetime
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter
from scipy.spatial import distance_matrix

In [ ]:
"""
import custom packages
"""
from module_.info.testbed_info import activityfiles_new
from module_.info.config import config, feature_name
from module_.readText import create_episodes, time_correction
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection
# from module_.info.testbed_info import d_files, g_files, c_files, p_files
# from module_.info.config import config, feature_name
# from module_.readText import create_episodes, time_correction, create_episodes_intra
# from module_.featureExtraction import feature_extraction
# from module_.changePointDetection import change_point_detection
# from module_.evaluation import episode_evaluation

In [ ]:
"""
load raw files
"""
dir_="dataset/testbed/npy/seminar/MS"
task_dict={i:[np.load("{}/{}".format(dir_, name)) for name in v] for i, v in enumerate(activityfiles_new.values())}
initial_dict={i:k[0] for i, k in enumerate(activityfiles_new.keys())}
label_dict={k[0]:k for k in activityfiles_new.keys()}

episodes, trs, tags = create_episodes(task_dict, initial_dict)
episodes=[time_correction(eps, trs[i]) for i, eps in enumerate(episodes)]

In [ ]:
"""
creating folders
"""

dataset='testbed'
preprocess='MS'
metric='RuLSIF'

for i, eps in enumerate(episodes):

    pairname="{}-{}".format(label_dict[tags[i][0]], label_dict[tags[i][2]])
    actpairfolder="./outputs/{}/{}/{}".format(dataset, preprocess, pairname)
    epsfolder="{}/{}_{}_{}".format(actpairfolder, i, tags[i], trs[i])
    metricfolder="{}/CPD/{}".format(epsfolder, metric)
    
    if not os.path.exists(actpairfolder):
        os.mkdir(actpairfolder)
    if not os.path.exists(epsfolder):
        os.mkdir(epsfolder)
    if not os.path.exists("{}/event".format(epsfolder)):
        os.mkdir("{}/event".format(epsfolder))
    if not os.path.exists("{}/feature".format(epsfolder)):
        os.mkdir("{}/feature".format(epsfolder))
    if not os.path.exists("{}/CPD".format(epsfolder)):
        os.mkdir("{}/CPD".format(epsfolder))
    if not os.path.exists(metricfolder):
        os.mkdir(metricfolder)

In [ ]:
dataset='testbed'
preprocess='MS'
metric='RuLSIF'

for i, eps in enumerate(episodes):
    print(i, tags[i])
    pairname="{}-{}".format(label_dict[tags[i][0]], label_dict[tags[i][2]])
    actpairfolder="./outputs/{}/{}/{}".format(dataset, preprocess, pairname)
    epsfolder="{}/{}_{}_{}".format(actpairfolder, i, tags[i], trs[i])
    metricfolder="{}/CPD/{}".format(epsfolder, metric)

    np.save("{}/event/eps.npy".format(epsfolder), eps)

    sensor_list=sorted(set(eps[:,0]))
    features=feature_extraction(eps, dataset, sensor_list)

    np.save("{}/feature/feat.npy".format(epsfolder), features)

    scores=change_point_detection(features, metricfolder, data_name=dataset, metric=metric, save=True)
    scores=np.array(scores)
    scores[scores<0]=0
    
###
    plt.title("{}-{}".format(tags[i], i))
    plt.ylabel('score')
    plt.xlabel('event')
    plt.ylim(0,3)
    plt.bar(range(len(eps)), scores)
    # plt.plot(positive, np.array(scores)[positive], 'bx', label='peak')
    # plt.axhline(y=0.1, linestyle=':', color='r', label='threshold')
    plt.axvline(x=trs[i], linestyle=':', color='g', label='transition')
    plt.legend()
    plt.savefig("{}/graph.png".format(metricfolder))
    plt.clf()
    
###

# Save events and features according to Transition/Non-transition

In [15]:
dataset='testbed'
preprocess='MS'
metric='RuLSIF'


threshold=60

for activitydir in glob.glob("./outputs/{}/{}/*".format(dataset, preprocess)):
    pairname=activitydir.split("/")[-1]
    leftset=set()
    rightset=set()
    for epsdir in glob.glob("{}/*".format(activitydir)):
        # epsnumber, actinitial, changepoint=epsdir.split("_")
        epsnumber, actinitial = epsdir.split("_")
        epsnumber=int(epsnumber.split("/")[-1])
        leftact, rightact = actinitial[1], actinitial[3]
        eps, changepoint=episodes[epsnumber], trs[epsnumber]
        features=np.load("{}/feature/feat.npy".format(epsdir))
        scores=np.load("{}/CPD/{}/scores.npy".format(epsdir, metric))

        Astarttime=float(eps[0,2])
        Aendtime=float(eps[changepoint-1,2])

        Bstarttime=float(eps[changepoint,2])
        Bendtime=float(eps[-1,2])

        TASe=np.array([item for item in eps[:changepoint] if float(item[2])<Astarttime+threshold])
        TAEe=np.array([item for item in eps[:changepoint] if float(item[2])>Aendtime-threshold])
        
        TBSe=np.array([item for item in eps[changepoint:] if float(item[2])<Bstarttime+threshold])
        TBEe=np.array([item for item in eps[changepoint:] if float(item[2])>Bendtime-threshold])

        NAe=np.array([item for item in eps[:changepoint] if Astarttime+threshold<=float(item[2])<=Aendtime-threshold])
        NBe=np.array([item for item in eps[changepoint:] if Bstarttime+threshold<=float(item[2])<=Bendtime-threshold])

        ####
        TASf=features[:TASe.shape[0]]
        TASs=scores[:TASe.shape[0]]
        assert len(TASf)==len(TASs)

        TAEf=features[changepoint-TAEe.shape[0]:changepoint]
        TAEs=scores[changepoint-TAEe.shape[0]:changepoint]
        assert len(TAEf)==len(TAEs)

        TBSf=features[changepoint:changepoint+TBSe.shape[0]]
        TBSs=scores[changepoint:changepoint+TBSe.shape[0]]
        assert len(TBSf)==len(TBSs)

        TBEf=features[len(features)-TBEe.shape[0]:]
        TBEs=scores[len(scores)-TBEe.shape[0]:]
        assert len(TBEf)==len(TBEs)

        NAf=features[len(TASf):changepoint-len(TAEf)]
        NAs=scores[len(TASs):changepoint-len(TAEs)]
        assert len(NAf)==len(NAs)

        NBf=features[changepoint+len(TBSf):len(features)-len(TBEf)]
        NBs=scores[changepoint+len(TBSs):len(scores)-len(TBEs)]
        assert len(NBf)==len(NBs)

        eventdir="{}/event".format(epsdir)
        featuredir="{}/feature".format(epsdir)
        scoredir="{}/CPD/{}/score".format(epsdir, metric)
        if not os.path.exists(scoredir):
            os.mkdir(scoredir)

        np.save("{}/TASe.npy".format(eventdir), TASe)
        np.save("{}/TBSe.npy".format(eventdir), TBSe)
        np.save("{}/TAEe.npy".format(eventdir), TAEe)
        np.save("{}/TBEe.npy".format(eventdir), TBEe)
        np.save("{}/NAe.npy".format(eventdir), NAe)
        np.save("{}/NBe.npy".format(eventdir), NBe)

        np.save("{}/TASf.npy".format(featuredir), TASf)
        np.save("{}/TBSf.npy".format(featuredir), TBSf)
        np.save("{}/TAEf.npy".format(featuredir), TAEf)
        np.save("{}/TBEf.npy".format(featuredir), TBEf)
        np.save("{}/NAf.npy".format(featuredir), NAf)
        np.save("{}/NBf.npy".format(featuredir), NBf)

        np.save("{}/TASs.npy".format(scoredir), TASs)
        np.save("{}/TBSs.npy".format(scoredir), TBSs)
        np.save("{}/TAEs.npy".format(scoredir), TAEs)
        np.save("{}/TBEs.npy".format(scoredir), TBEs)
        np.save("{}/NAs.npy".format(scoredir), NAs)
        np.save("{}/NBs.npy".format(scoredir), NBs)
    

In [27]:
data_name='testbed'
preprocess='MS'
metric='RuLSIF'

dict_={item:set() for item in ['C','T','G','S']}

for activitydir in glob.glob("./outputs/{}/{}/*".format(data_name, preprocess)):
    pairname=activitydir.split("/")[-1]

    for epsdir in glob.glob("{}/*".format(activitydir)):

        epsnumber, actinitial = epsdir.split("_")
        epsnumber=int(epsnumber.split("/")[-1])
        leftact, leftorder = actinitial[0], actinitial[1]
        rightact, rightorder = actinitial[2], actinitial[3]

        if leftorder in dict_[leftact]:
            continue
        dict_[leftact].add(leftorder)
        print(leftact, leftorder)
        # events
        TASe=np.load("{}/event/TASe.npy".format(epsdir))
        # TBSe=np.load("{}/event/TBSe.npy".format(epsdir))
        # TAEe=np.load("{}/event/TAEe.npy".format(epsdir))
        # TBEe=np.load("{}/event/TBEe.npy".format(epsdir))
        # NAe=np.load("{}/event/NAe.npy".format(epsdir))
        # NBe=np.load("{}/event/NBe.npy".format(epsdir))

        # features
        # TASf=np.load("{}/feature/TASf.npy".format(epsdir))
        # TBSf=np.load("{}/feature/TBSf.npy".format(epsdir))
        # TAEf=np.load("{}/feature/TAEf.npy".format(epsdir))
        # TBEf=np.load("{}/feature/TBEf.npy".format(epsdir))
        # NAf=np.load("{}/feature/NAf.npy".format(epsdir))
        # NBf=np.load("{}/feature/NBf.npy".format(epsdir))

        # scores
        # scoredir="{}/CPD/{}/score".format(epsdir, metric)
        # TASs=np.load("{}/TASs.npy".format(scoredir))
        # TBSs=np.load("{}/TBSs.npy".format(scoredir))
        # TAEs=np.load("{}/TAEs.npy".format(scoredir))
        # TBEs=np.load("{}/TBEs.npy".format(scoredir))
        # NAs=np.load("{}/NAs.npy".format(scoredir))
        # NBs=np.load("{}/NBs.npy".format(scoredir))

        # print(TASe.shape[0])

        idx_TASs=sorted([(score, idx) for idx, score in enumerate(TASs) if score<0.1], reverse=True)

        for score, index_ in idx_TASs:
            print(score)
            views=np.array([TASf[max(0, index_-1)], TASf[index_], TASf[min(len(TASf)-1, index_+1)]])
            # print(np.square(distance_matrix(before[:2], before[:2])), "test-test")
            # print(np.square(distance_matrix(before[:2], before[1:])), "test-train")
            # print(np.tril(distance_matrix(before, before)))
            A1A2=np.square(distance_matrix(views[:2], views[:2]))[0,1]
            A1A3=np.square(distance_matrix(views[:2], views[1:]))[0,1]
            A2A3=np.square(distance_matrix(views[:2], views[1:]))[1,1]
            print(A1A2, A1A3, A2A3)

S 1
0.08728875992158525
0.08515268158250278
0.049854075229224804
0.029804091391268828
0.02973459774246856
0.029080050189841833
0.02315303356239129
-0.054237297617473035
-0.06154835269767578
-0.06620915676863415
-0.0821033995439453
-1.903265993117241
-3.655443015205231
-4.886644349929233
S 0
0.08728875992158525
0.08515268158250278
0.049854075229224804
0.029804091391268828
0.02973459774246856
0.029080050189841833
0.02315303356239129
-0.054237297617473035
-0.06154835269767578
-0.06620915676863415
-0.0821033995439453
-1.903265993117241
-3.655443015205231
-4.886644349929233
T 1
0.08728875992158525
0.08515268158250278
0.049854075229224804
0.029804091391268828
0.02973459774246856
0.029080050189841833
0.02315303356239129
-0.054237297617473035
-0.06154835269767578
-0.06620915676863415
-0.0821033995439453
-1.903265993117241
-3.655443015205231
-4.886644349929233
T 0
0.08728875992158525
0.08515268158250278
0.049854075229224804
0.029804091391268828
0.02973459774246856
0.029080050189841833
0.0231530